In [9]:
summoner='5bP75FEeodLSJd1NcQlO9aL0G8vrjYiD8lAP1nw5X6yUtECy'
match = na_matches[0]
summoner in [summoner['player']['summonerId'] for summoner in match['participantIdentities']]

True

In [ ]:
player_id = 1
match['participants'][player_id]['timeline']

In [17]:
def get_stats(summoner='YKi_FDk4Sv8jgG35e6M_VnmZO8Dr4GMF0MSxS8Ga854BuPQ', matches = na_matches, timelines = na_timelines):
# For each player:
    # Main role = most played role where role is played > 40% of the time
    # Main champion = most played champion
    
    # Select all matches and timelines that they are in
    for i, match in enumerate(matches):
        is_in_game = summoner in [summoner['player']['summonerId'] for summoner in match['participantIdentities']]
        if is_in_game:
        # For each match:
        # Match info:
            game_id = match['gameId']
            player_id = int([player['participantId'] for player in match['participantIdentities'] 
                if player['player']['summonerId'] == summoner][0])
            # Identify champion played
            champ_played = match['participants'][player_id]['championId']
            # Identify role played
            role = match['participants'][player_id]['timeline']['role']
            # Identify lane opponent
            team = match['participants'][player_id]['teamId']
            opponent_id = int([player['participantId'] for player in match['participantIdentities'] 
                if match['participants'][player['participantId']-1]['timeline']['role'] == role
                               and match['participants'][player['participantId']-1]['team'] != team])    
            # Get CSD and XPD @ 15
            csd_10 = 
            # Get DMG share
            # Get DMG Taken share
            # Get 0-10 CSPM
            # Get 10-20 CSPM
            # Get 20+ CSPM:
                # Subtract CS at 20 from total CS, divide by game time 
            # Get vision score
            # Get overall kill participation
            # % of team's objective damage % of team's turret damage
            # of dragons killed / game
            # of barons killed / game average time to first baron

        # For each timeline:
            # Get number of duel kills and deaths
            # Get number of teamfight kills and deaths
            # Get number of skirmish kills and deaths
            # Get 0-10 K+D+A
            # Get 10-20 K+D+A
            # Get 20+ K+D+A
            # Get 0-10 wards placed
            # Get 10-20 wards placed
            # Get 20+ wards placed
            # Get wards cleared
            # Get pinks purchased
            # Get time of first kill/assist
            # get time of first tower
            # get number of solo kills
            # Get total Kills + Assists
            # STRETCH: get # of kills past enemy turret line
        

In [ ]:
get_stats()

In [ ]:
players_to_evaluate

In [ ]:
ids = []
with jsonlines.open(f'../data/scraped_ids_na.jsonl') as infile:
                for line in infile.iter():
                    ids.append(line)

Questions to answer:
*We want to use ratios not absolute stats, as we're comparing high-elo to low-elo so stuff like CSPM is just not going to be useful...*

#### Will need later:
- Most-played champion
- Most-played role

#### Fighting fingerprint: percentage of fights with are duels, skirmishes, and teamfights...
- Can I calculate duels (1v1 fights as detected by the death of one or both players)
- Stat contribution: easy. Get DMG Share and DMG Taken Share
- Can I calculate teamfights (3v3+ fights as detected by assists or kills)
- Can I calculate picks (1 player dying to 2 or more players with no return kill (how to define return kill?)
- Can I calculate skirmishes (2vX)

#### Farming to Fighting Ratio
Early Game: pre-10 CSPM / average(kills + assists + deaths) before 10  
Mid Game: 10-20 min CSPM / average(kills + assists + deaths) between 10 and 20  
Late Game: 20+ CSPM / average(kills + assists + deaths) after 20  

#### Lane aggressiveness:
CSD and XPD @ 15   
average (kills + assists + deaths) before 15    
(STRETCH) relate to number of roams?    

#### Vision focus:
Wards placed pre-10  
Wards placed pre-20  
Wards placed pre-30  

Vision Score/Minute   

Wards Cleared  
Pinks purchased  
(STRETCH) % of enemy wards cleared  

#### Aggression
Average time to First Kill/Assist  
Average time to First Tower  
% of kills = solo kills  
Forward Kills: % of kills past enemy 1st turret line  
Kill Participation  
Overall Kills+Assists / Minute  

#### Teamwork
% of team's objective damage
% of team's turret damage  
\# of dragons killed / game  
\# of barons killed / game
average time to first baron





In [15]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import jsonlines
import json

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [4]:
def ms_to_s(ms):
    return round(ms / 1000)

In [5]:
na_matches = []
with jsonlines.open('../data/cleaned/matches_na.jsonl') as infile:
    for line in infile:
        na_matches.append(line)
        
        

In [6]:
na_timelines = []
with jsonlines.open('../data/cleaned/timelines_na.jsonl') as infile:
    for line in infile:
        na_timelines.append(line)

In [7]:
all_players = []

for match in na_matches:
    for player in match['participantIdentities']:
        all_players.append(player['player']['summonerId'])

players_to_evaluate = pd.Series(all_players).value_counts().reset_index(name="count").query("count > 50")["index"]


In [ ]:
[[(i, event) for event in frame['events'] if event['type'] == 'CHAMPION_KILL' and event['killerId'] == 3] for i, frame in enumerate(ranked_timelines[0]['frames'])]

In [ ]:
ranked_timelines[0]['frames'][11]['events']

In [ ]:
# CSD and XPD @ 15   
# average (kills + assists + deaths) before 15    

ranked_matches[0]['participants']

We want to make sure we are incorporating only full games, where all players were connected and playing, so we will take a look at the distribution of maximum attained level in games which ended between 15 and 16 minutes (you can only initiate a surrender vote at the 15-minute mark). Based on this graph, we will not consider any games in which a player failed to exceed level 7 as valid.

In [ ]:
short_matches = [match for match in ranked_matches if match['gameDuration'] / 60 < 16 and match['gameDuration'] / 60 > 15]
levels = []
for match in short_matches:
    for participant in match['participants']:
        levels.append(participant['stats']['champLevel'])

plt.figure(figsize=(8,5))
sns.distplot(levels, bins=12, kde=False)
plt.xlabel('Max level attained')
plt.ylabel('Count')
plt.title('Distribution of Maximum Level Attained in 15-16 minute games')
plt.xticks(range(1,13));

In [ ]:
players_to_evaluate[0]

In [ ]:
all_players.head()

In [ ]:
ranked_matches[8]